In [ ]:
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
def SystematicResampling(W):
    Np = len(W[:,0])
    I = np.zeros(Np, dtype=int)
    u = stats.uniform(0,1/Np).rvs()
    for i in range(Np):
        acum_weigth, j, u_i = W[0,0], 0, u + (i-1)/Np
        while u_i > acum_weigth and j < Np:
            j += 1
            acum_weigth += W[j,0]
        I[i] = j 
    return I

In [ ]:
def MultinomialResampling(W):
    Np = len(W[:,0])
    return stats.multinomial(Np, W[:,0]).rvs()[0]

In [ ]:
def NoResampling(W):
    return np.array(range(len(W[:,0])))

In [ ]:
def SMC(Y, Np, Npop, p, p_obs, size, resampling_method = "multinomial"):
    # Set the resampling function
    if resampling_method == "multinomial":
        resampler = MultinomialResampling
    if resampling_method == "systematic":
        resampler = SystematicResampling
    else:
        resampler = NoResampling

    # Data length
    N = len(Y)
    # Array for particles
    Xp = np.zeros((Np, N, 2), dtype=int)
    # wp unnormalized weigths, Wp normalized weigths
    wp, Wp = np.zeros((Np, N)), np.zeros((Np, N))

    # Initial condition
    Xp[:,0,0] = Npop - stats.poisson(5).rvs()
    Xp[:,0,1] = Xp[:,0,0]

    # Negative binomial parameter p (probabily of success)
    param = size/(size + p_obs*Xp[:,0,1])

    # Weigths at first step
    wp[:,0] = stats.nbinom(size, param).pmf(Y[0]) 
    Wp[:,0] = wp[:,0]/np.sum(wp[:,0]) 

    # With resampling function generate an index array
    # to keep with high probability the particles with
    # higher weigths
    I = resampler(Wp)
    Xp = Xp[I,:,:]

    # Repeat for all the data
    for i in range(1, N):          
        Xp[:,i,1] = stats.binom(Xp[:,i-1,0], 1-(1-p)**Xp[:,i-1,1]).rvs(Np)
        Xp[:,i,0] = Xp[:,i-1,0] - Xp[:,i,1]
        param = size/(size + p_obs*Xp[:,i,1])
        wp[:,i] = stats.nbinom(size, param).pmf(Y[i])
        Wp[:,i] = wp[:,i]/np.sum(wp[:,i])
        I = resampler(Wp)
        Xp = Xp[I,:,:]

    return Xp, wp

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5e0fc8d6-78ad-425d-ae75-e4277a0eb228' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>